In [131]:
# Para llevar el registro
!pip install -q comet_ml
import comet_ml
comet_ml.init(project_name = 'FineTunning')

In [132]:
import pandas as pd
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Input, Conv2D, MaxPooling2D, Concatenate, PReLU
from tensorflow.keras.optimizers import RMSprop, SGD, Adam, Adadelta
from tensorflow.keras import regularizers
from keras.callbacks import ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.regularizers import l2

In [133]:
# Se crea el experimento en Comet
experiment = comet_ml.Experiment(
    auto_histogram_weight_logging = True,
    auto_histogram_gradient_logging = True,
    auto_histogram_activation_logging = True,
    log_code = True
)
experiment.set_name('Test 15')

COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Couldn't find a Git repository in '/kaggle/working' nor in any parent directory. Set `COMET_GIT_DIRECTORY` if your Git Repository is elsewhere.
COMET INFO: Experiment is live on comet.com https://www.comet.com/gurumasterahb/finetunning/486efeb756b143468f4c52cb905eea81



In [134]:
# Parámetros
i_width = int(178 * 1.2)
i_height = int(218 * 1.2)
input_shape = (i_width, i_height, 3)

parameters = {
    'num_class' : 2,
    'epochs' : 100,
    'batch_size' : 16,
    'learning_rate' : 0.0005,
    'loss' : 'binary_crossentropy',
    'optimizer' : 'RMSprop'}

experiment.log_parameters(parameters)

In [135]:
# Loading images
train_dir = '/kaggle/input/data-finetunning/Norma_align/Norma_align/train'
val_dir = '/kaggle/input/data-finetunning/Norma_align/Norma_align/val'

# Image Generator
gen_train = ImageDataGenerator(
    rotation_range = 30,
    width_shift_range = 0.15,  # % de desplazamiento horizontal
    height_shift_range = 0.15, # % de desplazamiento vertical
    brightness_range = [0.25, 1.25], # Porcentaje de brillo en el rango
    zoom_range = 0.3, # Zoom de hasta 30%
    channel_shift_range = 20, # Cada pixel de cada canal varía su intensidad en una vecindad de 20 unidades
    fill_mode = 'nearest', # Después de desplazamientos que dejan la imágen 'incompleta' se rellena con el valor de los pixeles más cercanos
    horizontal_flip = True,
    vertical_flip = True,
    rescale = 1./255.)
train = gen_train.flow_from_directory(
    train_dir,
    target_size = (i_width, i_height),
    color_mode = 'rgb',
    class_mode = 'binary',
    batch_size = parameters['batch_size'],
    shuffle = True)

gen_val = ImageDataGenerator(
    rotation_range = 30,
    width_shift_range = 0.15,
    height_shift_range = 0.15,
    brightness_range = [0.25, 1.25],
    zoom_range = 0.3,
    channel_shift_range = 20,
    fill_mode = 'nearest',
    horizontal_flip = True,
    vertical_flip = True,
    rescale = 1./255.)
val = gen_val.flow_from_directory(
    val_dir,
    target_size = (i_width, i_height),
    color_mode = 'rgb',
    class_mode = 'binary',
    batch_size = parameters['batch_size'],
    shuffle = True)

Found 120 images belonging to 2 classes.
Found 40 images belonging to 2 classes.


In [136]:
# Loading model
path_load = os.path.join('/kaggle/input/data-finetunning/best_val_loss.h5')
model_loaded = tf.keras.models.load_model(path_load)

model_0 = Sequential()

for layer in model_loaded.layers[:15]:
    layer.trainable = False
    model_0.add(layer)

# model_0.add(Dense(128, activation = 'relu', kernel_regularizer = l2(0.01)))
model_0.add(Dense(512, activation = 'relu'))
model_0.add(Dense(256, activation = 'relu'))
# model_0.add(Dropout(0.12))
model_0.add(Dense(128, activation = 'relu', kernel_regularizer = l2(0.0055)))
model_0.add(Dropout(0.5))
model_0.add(Dense(64, activation = 'relu', kernel_regularizer = l2(0.0055)))
model_0.add(Dense(1, activation = 'sigmoid'))
    
# model_0.summary()

In [137]:
# Checkpoint para el mejor modelo (val_accuracy)
filepath = 'best_val_loss.h5'
checkpoint = ModelCheckpoint(
    filepath,
    monitor = 'val_loss',
    verbose = 1,
    save_best_only = True,
    mode = 'min')

In [138]:
# Compilación y entrenamiento
model_0.compile(
    loss = parameters['loss'],
    optimizer = RMSprop(learning_rate = parameters['learning_rate']),
    metrics = ['accuracy'])

model_0.fit(
    train,
    epochs = parameters['epochs'],
    verbose = 1,
    validation_data = val,
    callbacks = [checkpoint])

Epoch 1/100
8/8 [==============================] - ETA: 0s - loss: 2.0570 - accuracy: 0.5250
Epoch 1: val_loss improved from inf to 2.05887, saving model to best_val_loss.h5


/opt/conda/lib/python3.10/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


8/8 [==============================] - 6s 467ms/step - loss: 2.0570 - accuracy: 0.5250 - val_loss: 2.0589 - val_accuracy: 0.5000
Epoch 2/100
8/8 [==============================] - ETA: 0s - loss: 1.9518 - accuracy: 0.5833
Epoch 2: val_loss improved from 2.05887 to 1.87168, saving model to best_val_loss.h5
8/8 [==============================] - 4s 484ms/step - loss: 1.9518 - accuracy: 0.5833 - val_loss: 1.8717 - val_accuracy: 0.6000
Epoch 3/100
8/8 [==============================] - ETA: 0s - loss: 1.8418 - accuracy: 0.5750
Epoch 3: val_loss improved from 1.87168 to 1.78173, saving model to best_val_loss.h5
8/8 [==============================] - 4s 521ms/step - loss: 1.8418 - accuracy: 0.5750 - val_loss: 1.7817 - val_accuracy: 0.6500
Epoch 4/100
8/8 [==============================] - ETA: 0s - loss: 1.7765 - accuracy: 0.6250
Epoch 4: val_loss improved from 1.78173 to 1.71699, saving model to best_val_loss.h5
8/8 [==============================] - 4s 457ms/step - loss: 1.7765 - accuracy:

/opt/conda/lib/python3.10/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


8/8 [==============================] - 4s 470ms/step - loss: 1.0825 - accuracy: 0.7667 - val_loss: 1.1802 - val_accuracy: 0.7250
Epoch 18/100
8/8 [==============================] - ETA: 0s - loss: 1.0492 - accuracy: 0.7750
Epoch 18: val_loss improved from 1.18017 to 1.13596, saving model to best_val_loss.h5
8/8 [==============================] - 3s 443ms/step - loss: 1.0492 - accuracy: 0.7750 - val_loss: 1.1360 - val_accuracy: 0.7250
Epoch 19/100
8/8 [==============================] - ETA: 0s - loss: 1.0395 - accuracy: 0.7833
Epoch 19: val_loss did not improve from 1.13596
8/8 [==============================] - 3s 426ms/step - loss: 1.0395 - accuracy: 0.7833 - val_loss: 1.2088 - val_accuracy: 0.6750
Epoch 20/100
8/8 [==============================] - ETA: 0s - loss: 0.9032 - accuracy: 0.8333
Epoch 20: val_loss did not improve from 1.13596
8/8 [==============================] - 3s 423ms/step - loss: 0.9032 - accuracy: 0.8333 - val_loss: 1.8719 - val_accuracy: 0.6000
Epoch 21/100
8/8 [==

In [139]:
experiment.log_model("Facial_r", filepath)
experiment.end()

COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.com/gurumasterahb/finetunning/486efeb756b143468f4c52cb905eea81
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     accuracy [100]                : (0.5249999761581421, 0.9083333611488342)
COMET INFO:     batch_accuracy [100]          : (0.25, 1.0)
COMET INFO:     batch_loss [100]              : (0.2584449052810669, 2.852083206176758)
COMET INFO:     epoch_duration [100]          : (2.7895452869997825, 6.077098996000132)
COMET INFO:     loss [100]                    : (0.3892442584037781, 2.0569818019866943)
COMET INFO:     val_accuracy [100]            : (0.5, 0.875)
COMET INFO:     val_loss [100]                : (0.479386895